
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# DEMO - Evaluation with Agent Framework

After building the model, it is essential to evaluate its performance. In this demo, we will demonstrate how to use the **Agent Framework** to assess the model's performance. The example will show how to calculate built-in evaluation metrics and define custom metrics.

**Learning Objectives:**

*By the end of this demo, you will be able to:*

* Load a model from the model registry for evaluation.

* Identify common built-in evaluation metrics.

* Define custom evaluation metrics.

* Run an evaluation test and view the results using the UI or code.

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.
   
   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **16.2.x-cpu-ml-scala2.12**



## Classroom Setup

Install required libraries.

In [0]:
%pip install -qq -U databricks-sdk langchain-databricks databricks-vectorsearch databricks-agents langchain==0.3.7 langchain-community==0.3.7
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-03


The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


## Prepare for Evaluation

The first step is to load the model that will be used for evaluation. The RAG model we built in the previous demo was **registered in the UC model registry**.

After loading the model, we will **define the evaluation dataset**. This dataset must include a **"request"** field to be used as the input query for the model. It typically also includes **"ground-truth" fields for response or context**. In this demo, to keep it simple, we will use only one request-response pair.



### Load the Model

Let's start by loading the model so that we can use it to generate responses and evaluate those responses later. Note that we will load a specific version of the model.

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")

model_uri = f"models:/{DA.catalog_name}.{DA.schema_name}.getstarted_genai_rag_demo/1"
print(model_uri)

models:/dbacademy.labuser10813094_1751496013.getstarted_genai_rag_demo/1


### Load Evaluation Dataset

In this demo we will only use **"request" and "expected_response"**. For examples of using other fields, you can check [sample evaluation sets](https://docs.databricks.com/en/generative-ai/agent-evaluation/evaluation-set.html#sample-evaluations-sets). 

Evaluation dataset is crucial for the accuracy of the evaluation. Some of the best practices are; 
* When developing an evaluation set, treat each sample as a unit test with a specific scenario and expected outcome, **including challenging examples and adversarial scenarios**. 

* Ensure the evaluation set **reflects a variety of questions**, even beyond the application's primary domain, to prevent hallucinations or harmful responses. 

* **High-quality, consistent human-generated labels are crucial**; achieve this by aggregating responses from multiple labelers and providing clear instructions to ensure consistency.

In [0]:
import pandas as pd

eval_set = pd.DataFrame([
    {
        "request": "personalized wooden name sign with app control",
        "expected_response": "Add a personal touch to your space with this handcrafted wooden name sign, perfect for nurseries, kids' rooms, weddings, or as a thoughtful gift. Made from high-quality wood, this custom sign is available in various sizes, fonts, and colors to suit your style. Lightweight and easy to install, it’s designed for durability and a smooth finish. Simply choose your size and color, enter the desired name or word, and let us create a unique piece just for you. Each sign is made to order with care, ensuring a timeless addition to your home or event decor.",
    }
])

## Model Evaluation

### Build-in Metrics

Mosaic AI Agent Framework uses LLM-judges to evaluate the model using common evaluation metrics such as safety and relevance. Also, it calculates system metrics such as token count and latency.

### Custom Metric

Although the Agents Evaluation framework automatically calculates common evaluation metrics, there are instances where we may need to assess the model using custom metrics. In this section, we will define a custom metric to evaluate whether the **response** generated is creative or not.

In [0]:
from mlflow.metrics.genai import make_genai_metric_from_prompt

# Define a custom assessment to detect PII in the retrieved chunks. 
creativity_prompt = "Your task is to determine whether generated product idea is creative or not. Grade idea from 1 to 5, 5 being very creative, 1 being very boring. This is the output: {response}."

creativity_level = make_genai_metric_from_prompt(
    name="creativity_level",
    judge_prompt=creativity_prompt,
    model="endpoints:/databricks-meta-llama-3-3-70b-instruct",
    metric_metadata={"assessment_type": "ANSWER"},
)

### Run Evaluation Test

Please note that in the code below, we are logging the evaluation process using MLflow to enable viewing the results through the MLflow UI.

In [0]:
print(model_uri)
with mlflow.start_run(run_name="rag_eval_demo_04_01"):
    eval_results = mlflow.evaluate(
        data=eval_set,
        model = model_uri,
        model_type = "databricks-agent",
        extra_metrics=[creativity_level]
    )

models:/dbacademy.labuser10813094_1751496013.getstarted_genai_rag_demo/1


2025/07/02 23:19:14 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating:   0%|          | 0/1 [Elapsed: 00:00, Remaining: ?]

/root/.ipykernel/2131/command-2179908554358456-30654587:17: LangChainDeprecationWarning: Use databricks_langchain.DatabricksVectorSearch


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results: 0/1 pass

## Review Evaluation Results

We have two options for reviewing the evaluation results. The first option is to examine the metrics and tables directly using the results object. The second option is to review the results through the user interface (UI).

### Review Results via the UI

To view the results in the UI, follow these steps:

- Click on the **"Experiment"** link displayed at the top of the previous code block's output for a simpler method.

- Alternatively, you can navigate to "Experiments" in the left panel and locate the experiment registered with the title of this notebook.

- View the overall metrics in the **Model Metrics** tab.

- Examine detailed results for each assessment in the **Evaluation Results** tab.

### Review Results Table

In [0]:
display(eval_results.metrics)

{'agent/latency_seconds/average': 2.883,
 'agent/total_token_count/average': 2216.0,
 'agent/total_input_token_count/average': 2121.0,
 'agent/total_output_token_count/average': 95.0,
 'response/llm_judged/safety/rating/percentage': 1.0,
 'response/llm_judged/creativity_level/rating/percentage': 1.0,
 'response/llm_judged/correctness/rating/percentage': 0.0,
 'response/llm_judged/groundedness/rating/percentage': 1.0,
 'retrieval/llm_judged/context_sufficiency/rating/percentage': 0.0,
 'response/overall_assessment/rating/percentage': 0.0}

In [0]:
display(eval_results.tables['eval_results'])

request_id request response retrieved_context trace tool_calls expected_response response/overall_assessment/rating response/overall_assessment/rationale response/llm_judged/safety/rating response/llm_judged/safety/rationale response/llm_judged/creativity_level/rating response/llm_judged/creativity_level/rationale response/llm_judged/correctness/rating response/llm_judged/correctness/rationale response/llm_judged/groundedness/rating response/llm_judged/groundedness/rationale retrieval/llm_judged/context_sufficiency/rating retrieval/llm_judged/context_sufficiency/rationale agent/latency_seconds agent/total_token_count agent/total_input_token_count agent/total_output_token_count fdc6bfd570de6136b244dc7aa23070eb10a063364f7ec43648329098f7d02e3a personalized wooden name sign with app control List(The provided context does not mention a "personalized wooden name sign with app control." The context appears to be related to Databricks, a data engineering platform, and discusses various topics such as Unity Catalog, Delta Sharing, and SQL commands. 

Therefore, I couldn't find any relevant information to answer the question about a "personalized wooden name sign with app control." If you have any further questions or need assistance with a different topic, please let me know.) List(List(## URL: https://docs.databricks.com/en/archive/dev-tools/cli/unity-catalog-cli.html

## Content: Recipients
Use the unity-catalog recipients subcommand to work with data recipients for Delta Sharing enabled Unity Catalog metastores. Data recipients are people or groups whom a Databricks user shares data with outside of the Databricks user’s organization. See Create and manage data recipients for Delta Sharing. 
To display usage documentation, run databricks unity-catalog recipients --help. 
Create a recipient 
Change a recipient’s settings 
Rotate a recipient’s token 
List available recipients 
List a recipient’s share permissions 
Get information about a recipient 
Delete a recipient 
Create a recipient 
To display usage documentation, run databricks unity-catalog recipients create --help. See also Create and manage data recipients for Delta Sharing. 
To generate a new activation URL for a recipient, run databricks unity-catalog recipients rotate-token. For an example, see Rotate a recipient’s token. 
databricks unity-catalog recipients create --name my-recipient 
Output: 
{ "name": "my-recipient", "created_at": 1656435288003, "created_by": "someone@example.com", "tokens": [ { "id": "12a345b6-7890-1cd2-3456-e789f0a12b34", "created_at": 1656435288005, "created_by": "someone@example.com", "activation_url": "https://example.com/delta_sharing/retrieve_config.html? ", "expiration_time": 1656521688006, "updated_at": 1656435288006, "updated_by": "someone@example.com" } ], "authentication_type": "TOKEN", "updated_at": 1656435288003, "updated_by": "someone@example.com", "owner": "someone@example.com" } 
Change a recipient’s settings 
To display usage documentation, run databricks unity-catalog recipients update --help. 
databricks unity-catalog recipients update --name my-recipient \ --json-file update-recipient-settings.json 
update-recipient-settings.json: 
{ "name": "my-recipient-2" } 
Output: 
{ "name": "my-recipient-2", "created_at": 1656435288003, "created_by": "someone@example.com", "tokens": [ { "id": "12a345b6-7890-1cd2-3456-e789f0a12b34", "created_at": 1656435288005, "created_by": "someone@example.com", "expiration_time": 1656436808507, "updated_at": 1656435908507, "updated_by": "someone@example.com" }, { "id": "12a345b6-7890-1cd2-3456-e789f0a12b34", "created_at": 1656435908509, "created_by": "someone@example.com", "activation_url": "https://example.com/delta_sharing/retrieve_config.html? ", "expiration_time": 1656522308510, "updated_at": 1656435908510, "updated_by": "someone@example.com" } ], "authentication_type": "TOKEN", "updated_at": 1656436740986, "updated_by": "someone@example.com", "owner": "someone@example.com" } 
Rotate a recipient’s token 


## Clean up Classroom

**🚨 Warning:** Please refrain from deleting resources created in this demo, as they are required for upcoming demonstrations. To clean up the classroom assets, execute the classroom clean-up script provided in the final demo.


## Summary

In this demo, we demonstrated how to load a model from the model registry and evaluate its performance using both built-in metrics and custom metrics. As an example of a custom metric, we defined one that assesses the creativity of the generated response. After running the evaluation, we showed how to view the results using the MLflow UI and the result object table.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>